In [25]:
#!/usr/bin/env python
#inspired by https://github.com/Matuzas77/MNIST-0.17/blob/master/MNIST_final_solution.ipynb
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'test'))

import numpy as np
from tinygrad.tensor import Tensor#, GPU
from tinygrad.nn import BatchNorm2D
from extra.utils import get_parameters
from datasets import fetch_mnist
from extra.training import train, evaluate, sparse_categorical_crossentropy
import tinygrad.nn.optim as optim
# from extra.augment import augment_img
GPU = os.getenv("GPU", None) is not None
QUICK = os.getenv("QUICK", None) is not None
DEBUG = os.getenv("DEBUG", None) is not None

class SqueezeExciteBlock2D:
    def __init__(self, filters):
        self.filters = filters
        self.weight1 = Tensor.uniform(self.filters, self.filters//32)
        self.bias1 = Tensor.uniform(1,self.filters//32)
        self.weight2 = Tensor.uniform(self.filters//32, self.filters)
        self.bias2 = Tensor.uniform(1, self.filters)

    def __call__(self, input):
        se = input.avg_pool2d(kernel_size=(input.shape[2], input.shape[3])) #GlobalAveragePool2D
        se = se.reshape(shape=(-1, self.filters))
        se = se.dot(self.weight1) + self.bias1
        se = se.relu()
        se = se.dot(self.weight2) + self.bias2
        se = se.sigmoid().reshape(shape=(-1,self.filters,1,1)) #for broadcasting
        se = input.mul(se)
        return se

class ConvBlock:
    def __init__(self, h, w, inp, filters=128, conv=3):
        self.h, self.w = h, w
        self.inp = inp
        #init weights
        self.cweights = [Tensor.uniform(filters, inp if i==0 else filters, conv, conv) for i in range(3)]
        self.cbiases = [Tensor.uniform(1, filters, 1, 1) for i in range(3)]
        #init layers
        self._bn = BatchNorm2D(128)#, training=True)
        self._seb = SqueezeExciteBlock2D(filters)

    def __call__(self, input):
        x = input.reshape(shape=(-1, self.inp, self.w, self.h))
        for cweight, cbias in zip(self.cweights, self.cbiases):
            x = x.pad2d(padding=[1,1,1,1]).conv2d(cweight).add(cbias).relu()
        x = self._bn(x)
        x = self._seb(x)
        return x

class BigConvNet:
    def __init__(self):
        self.conv = [ConvBlock(28,28,1), ConvBlock(28,28,128), ConvBlock(14,14,128)]
        self.weight1 = Tensor.uniform(128,10)
        self.weight2 = Tensor.uniform(128,10)

    def parameters(self):
        if DEBUG: #keeping this for a moment
            pars = [par for par in get_parameters(self) if par.requires_grad]
            no_pars = 0
            for par in pars:
                print(par.shape)
                no_pars += np.prod(par.shape)
            print('no of parameters', no_pars)
            return pars
        else:
            return get_parameters(self)

    def save(self, filename):
        with open(filename+'.npy', 'wb') as f:
            for par in get_parameters(self):
                #if par.requires_grad:
                np.save(f, par.cpu().data)

    def load(self, filename):
        with open(filename+'.npy', 'rb') as f:
            for par in get_parameters(self):
                #if par.requires_grad:
                try:
                    par.cpu().data[:] = np.load(f)
                    if GPU:
                        par.gpu()
                except:
                    print('Could not load parameter')

    def forward(self, x):
        x = self.conv[0](x)
        x = self.conv[1](x)
        x = x.avg_pool2d(kernel_size=(2,2))
        x = self.conv[2](x)
        x1 = x.avg_pool2d(kernel_size=(14,14)).reshape(shape=(-1,128)) #global
        x2 = x.max_pool2d(kernel_size=(14,14)).reshape(shape=(-1,128)) #global
        xo = x1.dot(self.weight1) + x2.dot(self.weight2)
        return xo.logsoftmax()

In [26]:
from pathlib import Path
from sklearn.model_selection import train_test_split

path_datasets = Path('./datasets')

ls = [f for f in path_datasets.glob('*') if f.is_file()]
print(ls)

[PosixPath('datasets/test.csv'), PosixPath('datasets/train.csv'), PosixPath('datasets/sample_submission.csv')]


In [27]:
_data = np.loadtxt(ls[1], skiprows=1, delimiter=',')

X = _data[:, 1:]
Y = _data[:,0]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
# x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

print(f'{X_train.shape=}')
print(f'{X_test.shape=}')
print(f'{Y_train.shape=}')
print(f'{Y_test.shape=}')

X_train.shape=(33600, 784)
X_test.shape=(8400, 784)
Y_train.shape=(33600,)
Y_test.shape=(8400,)


In [34]:
lrs = [1e-4, 1e-5] if QUICK else [1e-3, 1e-4, 1e-5, 1e-5]
epochss = [2, 1] if QUICK else [13, 3, 3, 1]
BS = 32

lmbd = 0.00025
lossfn = lambda out,y: sparse_categorical_crossentropy(out, y) + lmbd*(model.weight1.abs() + model.weight2.abs()).sum()
steps = len(X_train)//BS
np.random.seed(1337)
if QUICK:
    steps = 1
    X_test, Y_test = X_test[:BS], Y_test[:BS]

model = BigConvNet()

# if len(sys.argv) > 1:
#     try:
#         model.load(sys.argv[1])
#         print('Loaded weights "'+sys.argv[1]+'", evaluating...')
#         evaluate(model, X_test, Y_test, BS=BS)
#     except:
#         print('could not load weights "'+sys.argv[1]+'".')

if GPU:
    params = get_parameters(model)
    [x.gpu_() for x in params]

for lr, epochs in zip(lrs, epochss):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(1,epochs+1):
        #first epoch without augmentation
        # X_aug = X_train if epoch == 1 else augment_img(X_train)
        train(model, X_train, Y_train.astype('int'), optimizer, steps=steps, lossfn=lossfn, BS=BS)
        accuracy = evaluate(model, X_test, Y_test.astype('int'), BS=BS)
        # model.save(f'examples/checkpoint{accuracy * 1e6:.0f}')


loss 0.90 accuracy 0.81:  30%|███▎       | 320/1050 [1:38:21<3:44:23, 18.44s/it]


KeyboardInterrupt: 

In [31]:
sparse_categorical_crossentropy

<function extra.training.sparse_categorical_crossentropy(out, Y)>

In [38]:
params = get_parameters(model)
[x.gpu() for x in params]

AttributeError: 'Tensor' object has no attribute 'gpu'

In [41]:
dir(params[0])

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imatmul__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__ipow__',
 '__isub__',
 '__le__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pow__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_add',
 '_conv2d',
 '_ctx',
 '_expand',
 '_max',
 '_mul',
 '_pool2d',
 '_pow',
 '_reduce',
 '_reshape',
 '_softmax',
 '_sub',
 '_sum',
 'abs',
 'add',
 'arange',
 'assign',
 'avg_pool2d',
 'backward',
 'broadcasted',
 'cat',
 'clip',
 'conv2d',
 'cpu',
 'cpu_',
 'data',
 'deepwalk',
 'default',
 'default_',
 'detach',
 'device',
 'div',
 'dot',
 'dropout',
 'dtype',
 'elu',
 'exp',
